In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CALC CLASSIFIER USING MAMMOGRAM PATCHES IN BCDR
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v003_3'

In [2]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications.xception import Xception
base_model=Xception(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=ReLU()(x)
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 block4_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 14, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 14, 14, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 14, 14, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 14, 14, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 14, 14, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 14, 14, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

In [4]:
len(model.layers)

139

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=8

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('mass_ben_mal_class02/train0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 540 images belonging to 2 classes.
Found 126 images belonging to 2 classes.


In [7]:
#Callbacks
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mass_ben_mal_class_xception_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19680\1152529500.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/3
67/67 [==============================] - 75s 1s/step - loss: 0.5828 - accuracy: 0.7068 - val_loss: 0.4049 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 2/3
67/67 [==============================] - 69s 1s/step - loss: 0.5314 - accuracy: 0.7632 - val_loss: 0.4609 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/3
67/67 [==============================] - 69s 1s/step - loss: 0.4405 - accuracy: 0.8158 - val_loss: 0.4646 - val_accuracy: 0.7833 - lr: 0.0010


In [9]:
#Training stage 2
for layer in model.layers[:90]:
    layer.trainable=False
for layer in model.layers[90:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19680\81111685.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/10
67/67 [==============================] - 144s 2s/step - loss: 0.4472 - accuracy: 0.8177 - val_loss: 0.3599 - val_accuracy: 0.8583 - lr: 1.0000e-04
Epoch 2/10
67/67 [==============================] - 134s 2s/step - loss: 0.4214 - accuracy: 0.8102 - val_loss: 0.3550 - val_accuracy: 0.8583 - lr: 1.0000e-04
Epoch 3/10
67/67 [==============================] - 126s 2s/step - loss: 0.3998 - accuracy: 0.8327 - val_loss: 0.3187 - val_accuracy: 0.8417 - lr: 1.0000e-04
Epoch 4/10
67/67 [==============================] - 123s 2s/step - loss: 0.3644 - accuracy: 0.8534 - val_loss: 0.3913 - val_accuracy: 0.8083 - lr: 1.0000e-04
Epoch 5/10
67/67 [==============================] - 124s 2s/step - loss: 0.3764 - accuracy: 0.8515 - val_loss: 0.3973 - val_accuracy: 0.8250 - lr: 1.0000e-04
Epoch 6/10
67/67 [==============================] - 129s 2s/step - loss: 0.3527 - accuracy: 0.8421 - val_loss: 0.4021 - val_accuracy: 0.8250 - lr: 1.0000e-04
Epoch 7/10
67/67 [==============================] - 

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19680\2279829289.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/150
67/67 [==============================] - 275s 4s/step - loss: 0.3888 - accuracy: 0.8477 - val_loss: 0.4484 - val_accuracy: 0.8000 - lr: 1.0000e-05
Epoch 2/150
67/67 [==============================] - 249s 4s/step - loss: 0.3474 - accuracy: 0.8346 - val_loss: 0.4724 - val_accuracy: 0.7833 - lr: 1.0000e-05
Epoch 3/150
67/67 [==============================] - 252s 4s/step - loss: 0.3375 - accuracy: 0.8496 - val_loss: 0.4840 - val_accuracy: 0.7667 - lr: 1.0000e-05
Epoch 4/150
67/67 [==============================] - 254s 4s/step - loss: 0.3501 - accuracy: 0.8459 - val_loss: 0.4711 - val_accuracy: 0.7583 - lr: 1.0000e-05
Epoch 5/150
67/67 [==============================] - 258s 4s/step - loss: 0.3369 - accuracy: 0.8741 - val_loss: 0.4713 - val_accuracy: 0.7583 - lr: 1.0000e-05
Epoch 6/150
67/67 [==============================] - 254s 4s/step - loss: 0.3269 - accuracy: 0.8628 - val_loss: 0.5109 - val_accuracy: 0.7500 - lr: 1.0000e-05
Epoch 7/150
67/67 [===========================

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(valid_generator.classes, Y_pred)
prec=precision_score(valid_generator.classes, y_pred,pos_label=1)
rec=recall_score(valid_generator.classes, y_pred,pos_label=1)
spe=recall_score(valid_generator.classes, y_pred,pos_label=0)
f1s=f1_score(valid_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(valid_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)

Found 126 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19680\3759042783.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(valid_generator)


Confusion Matrix:
[[82  5]
 [12 27]]
Sensitivity:  0.6923076923076923
Specificity:  0.9425287356321839
Precision score: 0.84375
Accuracy: 0.8650793650793651
F1 score: 0.7605633802816902
MCC score: 0.6742405266866022
ROC AUC score: 0.93692897141173


In [13]:
#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

[[1.59414694e-01]
 [2.77643770e-01]
 [1.11738527e-02]
 [2.95620598e-03]
 [3.61684524e-03]
 [1.91597968e-01]
 [4.24531609e-01]
 [2.07460922e-04]
 [1.18822463e-01]
 [1.08486436e-01]
 [1.25214711e-01]
 [1.60809040e-01]
 [6.64828122e-02]
 [3.61375302e-01]
 [3.33145201e-01]
 [8.66320282e-02]
 [2.85792258e-02]
 [1.47408336e-01]
 [5.34862988e-02]
 [3.78614813e-02]
 [6.12348765e-02]
 [5.27827963e-02]
 [5.53921983e-02]
 [1.91858694e-01]
 [1.51087902e-03]
 [1.99991301e-01]
 [7.60871312e-03]
 [2.00739573e-03]
 [3.79313886e-01]
 [2.34237947e-02]
 [1.15014553e-01]
 [7.69282579e-02]
 [5.73709756e-02]
 [4.11043316e-03]
 [9.55423176e-01]
 [9.28665757e-01]
 [1.65413643e-04]
 [1.11392722e-01]
 [1.58588424e-01]
 [1.55663505e-01]
 [5.55570191e-03]
 [3.35033312e-02]
 [2.44365901e-01]
 [9.67138074e-03]
 [5.18379882e-02]
 [4.83547412e-02]
 [1.73300579e-02]
 [7.80741684e-04]
 [8.81914020e-01]
 [1.42149359e-01]
 [8.62661633e-04]
 [8.66195653e-04]
 [2.87275738e-03]
 [3.16931941e-02]
 [3.18800330e-01]
 [8.650089

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('mass_ben_mal_class02/test', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
spe=recall_score(test_generator.classes, y_pred,pos_label=0)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)

Found 169 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19680\3439002409.py:15: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator)


Confusion Matrix:
[[107   6]
 [ 20  36]]
Sensitivity:  0.6428571428571429
Specificity:  0.9469026548672567
Precision score: 0.8571428571428571
Accuracy: 0.8461538461538461
F1 score: 0.7346938775510204
MCC score: 0.6423650359046089
ROC AUC score: 0.8849557522123894


In [15]:
#Print estimation result for ensemble model
print(Y_pred)
print(test_generator.classes)

[[2.19204977e-01]
 [2.89667606e-01]
 [9.67970714e-02]
 [1.02585321e-02]
 [9.56841782e-02]
 [2.96707760e-04]
 [2.62613952e-01]
 [4.70688893e-03]
 [3.19033451e-02]
 [8.54365935e-04]
 [9.80120152e-03]
 [2.74901353e-02]
 [4.21713926e-02]
 [9.00877938e-02]
 [5.41234575e-02]
 [7.12397834e-03]
 [4.46693540e-01]
 [5.03940582e-01]
 [4.35833726e-03]
 [3.38044703e-01]
 [1.83784306e-01]
 [6.58842688e-03]
 [1.73335969e-02]
 [6.19687885e-02]
 [1.42666306e-02]
 [3.48356180e-03]
 [1.18303800e-03]
 [1.66295469e-02]
 [7.63176918e-01]
 [8.85193646e-02]
 [4.81288843e-02]
 [1.40036449e-01]
 [3.83778363e-02]
 [2.51033694e-01]
 [6.56750007e-03]
 [2.29617232e-04]
 [1.99840203e-01]
 [3.03869605e-01]
 [8.20209235e-02]
 [6.27377212e-01]
 [9.14957970e-02]
 [6.16046861e-02]
 [4.99640917e-03]
 [1.63519271e-02]
 [6.50337636e-02]
 [5.36317425e-03]
 [7.35390931e-02]
 [6.01560762e-03]
 [4.53414232e-01]
 [3.17102037e-02]
 [3.68691385e-02]
 [6.12402819e-02]
 [3.90251279e-02]
 [1.20831095e-03]
 [1.37567157e-02]
 [3.644433